In [8]:
import requests
import json
import time
import re

# -------------------------------------------------------------------------
# CONFIGURATION
# -------------------------------------------------------------------------
# PASTE YOUR KEYS HERE (Keep them inside the quotes)
CLIENT_ID = '0443b40a-5125-4241-b31b-b92c93122f54_a43ba522-ea21-497a-a7d0-27bc684143fe'
CLIENT_SECRET = 'DaGFcl18CyayKT9jE4Ol31H7P5RQy/Y7GrVcbqvm2QA='

# API Endpoints
TOKEN_ENDPOINT = "https://icdaccessmanagement.who.int/connect/token"

# UPDATED: We are now searching the 'MMS' (Mortality and Morbidity Statistics) linearization
# This ensures we get actual codes (like CA23) instead of just URIs.
API_ROOT = "https://id.who.int/icd/release/11/2024-01/mms/search"

# -------------------------------------------------------------------------
# STEP 1: Get the Access Token (OAuth2)
# -------------------------------------------------------------------------
def get_auth_token(client_id, client_secret):
    payload = {
        'client_id': client_id,
        'client_secret': client_secret,
        'grant_type': 'client_credentials',
        'scope': 'icdapi_access'
    }
    
    print("Attempting to get token...")
    # Token endpoint always uses POST
    response = requests.post(TOKEN_ENDPOINT, data=payload)
    
    if response.status_code == 200:
        print("✅ Success! Token received.")
        return response.json()['access_token']
    else:
        print(f"❌ Error getting token: {response.status_code}")
        print(response.text)
        return None

# -------------------------------------------------------------------------
# STEP 2: Search for a Disease (The Logic)
# -------------------------------------------------------------------------
def search_icd11(term, token):
    headers = {
        'Authorization': f'Bearer {token}',
        'Accept': 'application/json',
        'Accept-Language': 'en',
        'API-Version': 'v2'
    }
    
    # We use the search endpoint
    params = {
        'q': term,           # The search term (e.g., "Asthma")
        'useflexisearch': 'true', # Fuzzy search
        'flatResults': 'false'    # We want hierarchical results if possible
    }
    
    print(f"🔍 Searching ICD-11 (MMS) for: '{term}'...")
    # FIXED: The Search endpoint expects a GET request, not POST
    response = requests.get(API_ROOT, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ API Error: {response.status_code}")
        return None

# Helper to remove HTML tags like <em class='found'> from results
def clean_html(raw_html):
    cleanr = re.compile('<.*?>')
    return re.sub(cleanr, '', raw_html)

# -------------------------------------------------------------------------
# MAIN EXECUTION
# -------------------------------------------------------------------------
if __name__ == "__main__":
    # 1. Authenticate
    token = get_auth_token(CLIENT_ID, CLIENT_SECRET)
    
    if token:
        # 2. Test Search (e.g., Asthma)
        search_term = "Asthma" 
        results = search_icd11(search_term, token)
        
        if results:
            print("\n--- 🎯 Search Results ---")
            # Parse and print just the relevant parts (Title and Code)
            for entity in results.get('destinationEntities', [])[:5]:
                title = clean_html(entity.get('title')) # Clean the HTML tags
                code = entity.get('theCode', 'No Code')
                uri = entity.get('id')
                print(f"Code: {code} | Disease: {title}")
                print(f"Link: {uri}\n")
            
            print("-------------------------")
            print("✅ If you see results above, your API integration is working!")

Attempting to get token...
✅ Success! Token received.
🔍 Searching ICD-11 (MMS) for: 'Asthma'...

--- 🎯 Search Results ---
Code: 1B10.Z | Disease: Respiratory tuberculosis, without mention of bacteriological or histological confirmation
Link: http://id.who.int/icd/release/11/2024-01/mms/882244568/unspecified

Code: 1F2Z | Disease: Mycoses, unspecified
Link: http://id.who.int/icd/release/11/2024-01/mms/1939815950/unspecified

Code: BD11.Z | Disease: Left ventricular failure, unspecified
Link: http://id.who.int/icd/release/11/2024-01/mms/437702716/unspecified

Code: CA0H.4 | Disease: Laryngeal spasm
Link: http://id.who.int/icd/release/11/2024-01/mms/1531260827

Code: CA22.1 | Disease: Certain specified chronic obstructive pulmonary disease
Link: http://id.who.int/icd/release/11/2024-01/mms/143605159

-------------------------
✅ If you see results above, your API integration is working!
